In [3]:
import io
import os
import shutil

output_dir = 'l2m'

In [4]:
SEP = '[sep]'
ALIGN = '[align]'
beam = 5

In [5]:
input_lyric_file = '/home/dwj/data/code/model_modify/zhonghao/songmass/data/test.lyric'
generate_melody_file = '/home/dwj/data/code/model_modify/zhonghao/songmass/generate/2021_7_7_test.log'
generated_file = True
get_last = True
find_structure = True
cut_exceed_sent = True

In [6]:
def get_pitch_duration_structure(note_seq):
    seq = []
    i = 0
    while (i<len(note_seq)):
        if note_seq[i] > 128:
            #Duration
            i += 1
            continue
        else:
            #Pitch
            if i+1>=len(note_seq):
                #No Duration Followed
                break
            if note_seq[i+1] <= 128:
                #Followed by a pitch
                i += 1
                continue
            
            #Followed by a duration
#             pitch = note_seq[i]
#             duration = float(Duration_vocab[note_seq[i+1]])
            
            #Here trans back to str for bleu calculate
            pitch = str(note_seq[i])
            duration = str(note_seq[i+1])
            
            seq.append(pitch)
            seq.append(duration)
            i += 2
    return seq

In [7]:
def separate_sentences(x,find_structure = False):
    lst = x.copy()
    sep_positions = [i for i,x in enumerate(lst) if x==SEP]
    sep_positions.insert(0,-1)

    ret = []
    for i in range(len(sep_positions)-1):
        sent = lst[sep_positions[i]+1:sep_positions[i+1]] #SZH: not include sep token
        if find_structure:
            sent = list(map(int, sent))
            sent = get_pitch_duration_structure(sent)
        ret.append(sent)
    return ret

In [8]:
def get_songs(file, generated_file = False, get_last = False, find_structure = False, cut_exceed_sent=False):
    """
    Get Last : Whether include the last tokens if the sequence not ends with a seperation token
    """
    with io.open(input_lyric_file,'r') as f:
        input_lyrics= f.readlines()
    input_lyrics = list(map(lambda x:x.rstrip('\n').split(' '), input_lyrics))
    input_lyrics_sent_num = list(map(lambda x:x.count(SEP), input_lyrics))
    
    with io.open(file,'r') as f:
        melody_lines = f.readlines()
        if generated_file:
            melody_lines = list(filter(lambda x:x.startswith('H-'), melody_lines))
            if len(melody_lines) == len(input_lyrics) * beam:
                melody_lines.sort( key=lambda x:(int(x.split('\t')[0].split('-')[1]), -float(x.split('\t')[1]) ) )
                melody_lines = [ x for i,x in enumerate(melody_lines) if i%beam == 0 ]
            else:    
                melody_lines.sort( key=lambda x:int(x.split('\t')[0].split('-')[1]) )
    melody_lines = list(map(lambda x:x.rstrip('\n').split('\t')[-1] ,melody_lines))
            
    print(len(melody_lines), len(input_lyrics))
    assert len(melody_lines)==len(input_lyrics)
    
    melody_seqs = list(map(lambda x:x.rstrip('\n').split(' '),melody_lines))
    melody_seqs = [ch for ch in melody_seqs if ch != ALIGN]
    
    for i in range(len(melody_seqs)):
        melody_seqs[i] = list(filter(lambda x:x.isdigit() or x==SEP, melody_seqs[i]))
        
    if get_last:
        for i in range(len(melody_seqs)):
            if melody_seqs[i][-1] != SEP:
                melody_seqs[i].append(SEP)
          
    # 分句子，同时find structure
    melody_seq_sents = list(map(lambda x:separate_sentences(x,find_structure=find_structure), melody_seqs))

    
    # 把句子组合回整首歌，同时切除过多的句子
    return_list = []
    for i,sent_seq in enumerate(melody_seq_sents):
        if cut_exceed_sent and len(sent_seq) > input_lyrics_sent_num[i]:
            sent_seq = sent_seq[0:input_lyrics_sent_num[i]]
        cur_song_return = []

        for j,sent in enumerate(sent_seq):
            cur_song_return.extend(sent)
            cur_song_return.append(SEP)
        return_list.append(cur_song_return)
    return return_list
            


In [9]:
output_melodies = get_songs(generate_melody_file, generated_file, get_last, find_structure, cut_exceed_sent)

714 714


In [10]:
with io.open(input_lyric_file,'r') as f:
    input_lyrics= f.readlines()
input_lyrics = list(map(lambda x:x.rstrip('\n').split(' '), input_lyrics))
input_lyrics_sent_num = list(map(lambda x:x.count(SEP), input_lyrics))

output_melodies_sent_num = list(map(lambda x:x.count(SEP), output_melodies))
list(zip(input_lyrics_sent_num, output_melodies_sent_num))

[(22, 22),
 (25, 25),
 (21, 21),
 (4, 4),
 (32, 32),
 (28, 28),
 (6, 6),
 (8, 8),
 (9, 9),
 (23, 23),
 (15, 15),
 (3, 3),
 (23, 23),
 (6, 6),
 (10, 10),
 (4, 4),
 (20, 20),
 (10, 10),
 (18, 18),
 (1, 1),
 (3, 3),
 (32, 32),
 (18, 18),
 (7, 7),
 (1, 1),
 (1, 1),
 (29, 29),
 (31, 31),
 (21, 21),
 (9, 9),
 (6, 6),
 (20, 20),
 (30, 30),
 (15, 15),
 (8, 8),
 (5, 5),
 (7, 7),
 (19, 19),
 (19, 19),
 (19, 19),
 (5, 5),
 (30, 30),
 (13, 13),
 (28, 28),
 (12, 12),
 (12, 12),
 (32, 32),
 (5, 5),
 (24, 24),
 (4, 4),
 (10, 10),
 (20, 20),
 (27, 27),
 (29, 29),
 (13, 13),
 (20, 20),
 (20, 20),
 (20, 20),
 (23, 23),
 (1, 1),
 (27, 27),
 (28, 28),
 (16, 16),
 (2, 2),
 (22, 22),
 (12, 12),
 (4, 4),
 (26, 26),
 (20, 20),
 (20, 20),
 (1, 1),
 (23, 23),
 (10, 10),
 (7, 7),
 (18, 18),
 (16, 16),
 (35, 35),
 (40, 40),
 (9, 9),
 (20, 20),
 (30, 30),
 (3, 3),
 (7, 7),
 (27, 27),
 (2, 2),
 (3, 3),
 (1, 1),
 (16, 16),
 (21, 21),
 (11, 11),
 (27, 27),
 (19, 19),
 (7, 7),
 (5, 5),
 (11, 11),
 (8, 8),
 (23, 23),
 

In [11]:
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

In [12]:
for i,x in enumerate(zip(input_lyrics,output_melodies)):
    cur_folder = os.path.join(output_dir, str(i))
    print(cur_folder)
    os.makedirs(cur_folder)
    with io.open(os.path.join(cur_folder,'train.lyric'),'w') as f:
        f.write(" ".join(x[0]))
    with io.open(os.path.join(cur_folder,'train.melody'),'w') as f:
        f.write(" ".join(x[1]))

/home/dwj/data/code/model_modify/zhonghao/not_pitch_normalization/evaluate/calculate_ppl/data/l2m/0
/home/dwj/data/code/model_modify/zhonghao/not_pitch_normalization/evaluate/calculate_ppl/data/l2m/1
/home/dwj/data/code/model_modify/zhonghao/not_pitch_normalization/evaluate/calculate_ppl/data/l2m/2
/home/dwj/data/code/model_modify/zhonghao/not_pitch_normalization/evaluate/calculate_ppl/data/l2m/3
/home/dwj/data/code/model_modify/zhonghao/not_pitch_normalization/evaluate/calculate_ppl/data/l2m/4
/home/dwj/data/code/model_modify/zhonghao/not_pitch_normalization/evaluate/calculate_ppl/data/l2m/5
/home/dwj/data/code/model_modify/zhonghao/not_pitch_normalization/evaluate/calculate_ppl/data/l2m/6
/home/dwj/data/code/model_modify/zhonghao/not_pitch_normalization/evaluate/calculate_ppl/data/l2m/7
/home/dwj/data/code/model_modify/zhonghao/not_pitch_normalization/evaluate/calculate_ppl/data/l2m/8
/home/dwj/data/code/model_modify/zhonghao/not_pitch_normalization/evaluate/calculate_ppl/data/l2m/9
